## Export Inventory CSV from the FairNow APIs

#### This generates a CVS export of the Inventory report, containing application, vendor, status, and risk level information.

### Prerequisites

#### To use this notebook, you'll need a `Client ID` and `Client Secret`. These will either have been provided to you, or you can generate from https://app.fairnow.ai and going the the Admin menu. This notebook assumes you have these available to enter when prompted.

In [2]:
import pandas as pd
from utils.fairnow import create_df, export_to_tsv, get_client

client = get_client(client_id='2726p9mj2ghit00n23tkjatp84') # Replace with your Client Id

## Create Applications DataFrame
#### Graphs can be created using this DataFrame output

In [ ]:
# Retrieve data from the API
application_route = "/applications"

apps_response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        apps_response = response.json()
        print(f"Successfully retrieved applications")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)

# Extract fields from response
extracted_data = []
for app in apps_response['applications']:
    app_id = app['application_id']
    app_name = app['application_name']
    risk_level = app.get('risk_level', '')
    vendor_id = app.get('vendor_id', '')
    extracted_data.append({
        'application_id': app_id,
        'application_name': app_name,
        'risk_level': risk_level,
        'vendor_id': vendor_id,
    })

# Convert to DataFrame
apps_df = create_df(extracted_data)


## Create Vendors DataFrame
#### Graphs can be created using this DataFrame output

In [ ]:
# Retrieve data from the API
application_route = "/vendors/"

response = None
try:
    response = client.get(application_route, timeout=None)
    if response.status_code == 200:
        response = response.json()
        print(f"Successfully retrieved vendors")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(e)

# Extract fields from response
extracted_data = []
for vendor in response:
    # Get vendor information
    vendor_id = vendor.get('vendor_id', '')
    vendor_name = vendor.get('vendor_name', '')
    status = vendor.get('status', '')
    
    # Get governance information
    governance = vendor.get('governance', {})
    risk_program = governance.get('risk_program', '') if governance else ''
    
    # Get linked applications
    linked_apps = vendor.get('linked_applications', [])
    # Only include vendors with linked applications
    if linked_apps:
        for linked_app in linked_apps:
            extracted_data.append({
                'application_id': linked_app.get('application_id', ''),
                'vendor_id': vendor_id,
                'vendor_name': vendor_name,
                'status': status,
                'risk_program': risk_program
            })

# Convert to DataFrame
vendors_df = create_df(extracted_data)

## Merge DataFrames
#### Graphs can be created using this DataFrame output

In [ ]:
# Merge DataFrames
merged_df = pd.merge(
    apps_df[['application_id', 'application_name', 'risk_level', 'vendor_id']],
    vendors_df[['vendor_id', 'vendor_name', 'status', 'risk_program']],
    on='vendor_id',
    how='left'
)
merged_df = merged_df.drop_duplicates()

print(f"Extracted {len(merged_df)} records")

## Export to TSV

In [ ]:
export_to_tsv(merged_df, 'inventory_export.tsv')